<a href="https://colab.research.google.com/github/Azimoj/WebScraping/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtenir les derniers articles du Monde (**)

Obtenir le titre et le chapeau (description) des derniers articles sur https://www.lemonde.fr/actualite-en-continu/ (*)

Enregistrer les images correspondantes (***, @)

In [4]:
import requests
from bs4 import BeautifulSoup

 Obtenir le titre et les chapeaux

La première étape est de demander la page au serveur en utilisant la fonction requests.get.

In [5]:
page= requests.get("https://www.lemonde.fr/actualite-en-continu/")

In [6]:
page.status_code

200

Noter le code de réponse : 200 = Success.

On peut afficher le contenu HTML de la page avec l'attribut content.

In [ ]:
page.content

In [ ]:
soup = BeautifulSoup(page.content)

print(soup.prettify()) # pour un meilleur affichage

On peut ensuite extraire le texte depuis les tags qui nous intéressent.

Pour connaître le noms des tags pertinents, on utilise l'inspecteur du navigateur. On va se concentrer sur la section id="river", qui contient les articles :

In [ ]:
content = soup.find('section', {'id':'river'})
content

In [12]:
# Syntaxe alternative : la méthode select

content2 = soup.select("html body section#river .teaser")

# Cette méthode utilise la syntaxe CSS pour rechercher un tag.
# Elle est parfois plus rapide à écrire. Les tags successifs sont
# separés par des espaces, les # désignent une class et les . un id.
# On peut obtenir le chemin CSS avec clic droit dans l'inspecteur.

On isole les titres (h3) et les chapeaux (p, class='teaser__desc') des articles :

In [17]:
titres = content.find_all('h3')
titres

[<h3 class="teaser__title">Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville</h3>,
 <h3 class="teaser__title">Ilia Iachine, opposant russe : « La véritable Russie ne combat pas les Ukrainiens sur le front. La véritable Russie est en prison »</h3>,
 <h3 class="teaser__title">Affaire Benalla : peine confirmée en appel pour Alexandre Benalla après les « violences du 1ᵉʳ-Mai »</h3>,
 <h3 class="teaser__title">Paris 2024 : les Russes ne sont pas exclus des Jeux paralympiques</h3>,
 <h3 class="teaser__title">L’inflation en France reste à un niveau élevé, la consommation des ménages en baisse, rapporte l’Insee</h3>,
 <h3 class="teaser__title">Epic Games, société créatrice du jeu vidéo « Fortnite », annonce le licenciement de plus de 800 personnes</h3>,
 <h3 class="teaser__title">Les Etats-Unis vont-ils connaître un nouveau « shutdown » ? Ce qu’il faut savoir sur cette bataille budgétaire et ses conséquences</h3>,
 <h3 class="teaser__title">Plus de 50 morts e

In [16]:
chapeaux = content.find_all('p', {'class': 'teaser__desc'})
chapeaux

[<p class="teaser__desc">L’engouement pour le nouveau traitement est tel que les commandes passées par les officines tardent à arriver. La direction générale de la santé a recommandé, vendredi 29 septembre, aux médecins de ville de suspendre les prescriptions du dosage pour les bébés de plus de 5 kilos.</p>,
 <p class="teaser__desc">L’homme politique membre de l’opposition, condamné à huit ans et demi de colonie pénitentiaire pour avoir dénoncé l’agression russe en Ukraine, a envoyé un texte à l’occasion d’un forum organisé par l’association Russie-Libertés, à Paris, les 29 et 30 septembre. « Le Monde » en publie de larges extraits.</p>,
 <p class="teaser__desc">En première instance, l’ancien collaborateur de l’Elysée avait été condamné à trois ans de prison, dont un an ferme sous bracelet électronique.</p>,
 <p class="teaser__desc">Le comité international paralympique a voté contre l’exclusion vendredi du comité russe. Une décision qui marque un infléchissement de la fermeté dont a fa

In [18]:
titres[0].get_text()

'Bronchiolite\xa0: la distribution du Beyfortus interrompue dans les pharmacies de ville'

Le package unidecode permet de décoder les caractères unicode pour nettoyer le texte obtenu :

In [19]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.1 MB/s eta 0:00:00


In [20]:
from unidecode import unidecode

unidecode('Attentats de 2016 à Bruxelles\xa0: Salah Abdeslam et neuf\xa0autres inculpés renvoyés aux assises')

'Attentats de 2016 a Bruxelles : Salah Abdeslam et neuf autres inculpes renvoyes aux assises'

In [ ]:
# Nettoyage sous forme de boucle
titres_clean = []
for t in titres:
    titres_clean.append( unidecode(t.get_text()) )
titres_clean

In [23]:
# Nettoyage sous forme de list comprehension (mieux)
titres = [ unidecode(t.get_text()) for t in titres ]
titres

['Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville',
 'Ilia Iachine, opposant russe : << La veritable Russie ne combat pas les Ukrainiens sur le front. La veritable Russie est en prison >>',
 'Affaire Benalla : peine confirmee en appel pour Alexandre Benalla apres les << violences du 1er-Mai >>',
 'Paris 2024 : les Russes ne sont pas exclus des Jeux paralympiques',
 "L'inflation en France reste a un niveau eleve, la consommation des menages en baisse, rapporte l'Insee",
 'Epic Games, societe creatrice du jeu video << Fortnite >>, annonce le licenciement de plus de 800 personnes',
 "Les Etats-Unis vont-ils connaitre un nouveau << shutdown >> ? Ce qu'il faut savoir sur cette bataille budgetaire et ses consequences",
 "Plus de 50 morts et plusieurs dizaines de blesses au Pakistan dans une explosion pres d'une mosquee",
 "Comment gerer les sans-abri toujours plus nombreux dans l'ouest des Etats-Unis ? Les elus sous pression pour trouver des solutions ",


In [24]:
chapeaux = [unidecode(t.get_text()) for t in chapeaux]
chapeaux

["L'engouement pour le nouveau traitement est tel que les commandes passees par les officines tardent a arriver. La direction generale de la sante a recommande, vendredi 29 septembre, aux medecins de ville de suspendre les prescriptions du dosage pour les bebes de plus de 5 kilos.",
 "L'homme politique membre de l'opposition, condamne a huit ans et demi de colonie penitentiaire pour avoir denonce l'agression russe en Ukraine, a envoye un texte a l'occasion d'un forum organise par l'association Russie-Libertes, a Paris, les 29 et 30 septembre. << Le Monde >> en publie de larges extraits.",
 "En premiere instance, l'ancien collaborateur de l'Elysee avait ete condamne a trois ans de prison, dont un an ferme sous bracelet electronique.",
 "Le comite international paralympique a vote contre l'exclusion vendredi du comite russe. Une decision qui marque un inflechissement de la fermete dont a fait preuve jusque-la l'IPC.",
 "Les prix a la consommation ont augmente de 4,9 % sur un an en septem

Pour terminer, on peut éventuellement stocker les résultats dans un dataframe :

In [25]:
import pandas as pd

In [26]:
# Création du df à partir d'un dictionnaire
dico = {'titre':titres, 'chapeau':chapeaux}
pd.DataFrame(dico)

,titre,chapeau
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [27]:
# Création du df à partir d'une liste
df = pd.DataFrame([titres, chapeaux]).T
df.columns = ['titre', 'chapeau']
df

,titre,chapeau
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [28]:
# Utilisation de zip
list(zip(titres, chapeaux))

[('Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville',
  "L'engouement pour le nouveau traitement est tel que les commandes passees par les officines tardent a arriver. La direction generale de la sante a recommande, vendredi 29 septembre, aux medecins de ville de suspendre les prescriptions du dosage pour les bebes de plus de 5 kilos."),
 ('Ilia Iachine, opposant russe : << La veritable Russie ne combat pas les Ukrainiens sur le front. La veritable Russie est en prison >>',
  "L'homme politique membre de l'opposition, condamne a huit ans et demi de colonie penitentiaire pour avoir denonce l'agression russe en Ukraine, a envoye un texte a l'occasion d'un forum organise par l'association Russie-Libertes, a Paris, les 29 et 30 septembre. << Le Monde >> en publie de larges extraits."),
 ('Affaire Benalla : peine confirmee en appel pour Alexandre Benalla apres les << violences du 1er-Mai >>',
  "En premiere instance, l'ancien collaborateur de l'Elysee ava

Note : jusqu'à maintenant, on a raisonné en cherchant tous les titres, puis tous les chapeaux, puis en les rassemblant. On aurait pu raisonner autrement, avec une approche plus "bottom-up" : chercher le titre et le chapeau d'un seul article, puis utiliser une boucle pour appliquer les calculs à tous les articles.

In [29]:
# Approche "bottom-up" : boucle pour extraire les infos de chaque article

articles = []
for article in content:
    try:
        title = article.find('h3').get_text()
        title = unidecode(title)
        chapeau = article.find('p').get_text(strip=True)
        chapeau = unidecode(chapeau)
        articles.append([title, chapeau])
    except:
        continue

pd.DataFrame(articles, columns=['titres', 'chapeaux'])

,titres,chapeaux
0,Bronchiolite : la distribution du Beyfortus in...,L'engouement pour le nouveau traitement est te...
1,"Ilia Iachine, opposant russe : << La veritable...","L'homme politique membre de l'opposition, cond..."
2,Affaire Benalla : peine confirmee en appel pou...,"En premiere instance, l'ancien collaborateur d..."
3,Paris 2024 : les Russes ne sont pas exclus des...,Le comite international paralympique a vote co...
4,"L'inflation en France reste a un niveau eleve,...","Les prix a la consommation ont augmente de 4,9..."
5,"Epic Games, societe creatrice du jeu video << ...",Epic Games a revele jeudi la preparation d'un ...
6,Les Etats-Unis vont-ils connaitre un nouveau <...,Les lois de depenses permettant de financer le...
7,Plus de 50 morts et plusieurs dizaines de bles...,"Le ministre de l'information du Balouchistan, ..."
8,Comment gerer les sans-abri toujours plus nomb...,La canicule a mis a jour le nombre croissant d...
9,Les punaises de lit : comment se propagent-ell...,Alors que la Mairie de Paris parle d'une << re...


In [30]:

len(articles)

60


 ### **Télécharger les images (***, @)

On commence par rechercher toutes les images de la section river :

In [ ]:
images = soup.find_all('img', src=True)
images

In [ ]:
# Syntaxe CSS
images = soup.select("html body section#river img[src^=http]")
images

On extrait les liens, avec la sélection d'un attribut du tag (attribut src) :

In [ ]:
links = [ link['src'] for link in images ]
links

Et enfin on enregistre les images grâce à ces liens. La syntaxe pour enregistrer des images utilise with open(), et elle n'est pas forcément intuitive :

In [35]:
import os
os.makedirs('images') # créer le dossier images

for i, link in enumerate(links):
    with open(f'images/my_image_{i}.jpg', 'wb') as dest:
        img = requests.get(link)
        dest.write( img.content )

Le beau temps fait-il le bonheur ? (**)
Les gens sont-ils plus heureux dans les pays ensoleillés ? Y a-t-il un lien entre le taux de dépression et le manque de soleil ? Intéressons-nous aux données pour répondre à cette question.


Hypothèse :

Moins un pays compte d'heures d'ensoleillement, plus le taux d'individus déprimés est élevé.

Data à collecter :

Noms de pays
Taux de personnes déprimées
Moyenne des heures d'ensoleillement
Ressources à utiliser :

https://en.wikipedia.org/wiki/Epidemiology_of_depression
https://en.wikipedia.org/wiki/List_of_cities_by_sunshine_duration
Questions :

Scraper les 2 tables (taux d'individus déprimés et heures d'ensoleillement). (**)
Nettoyer les données et calculer les moyennes annuelles d'ensoleillement par pays. (**)
Fusionner les tables. (*)
Afficher la corrélation à l'aide d'un scatter plot (sns.scatterplot). (*)

INDICES :

Dans cet exercice, nous scrapons des tables sur un site web. Pour cette tâche, on peut utiliser BeautifulSoup comme d'habitude, ou utiliser la fonction pandas pd.read_html() qui est plus rapide (meilleure solution ici). Après avoir scrapé les données des articles, il reste simplement à les fusionner et à étudier la corrélation.


 Dépression

Commençons par regarder les taux de personnes déprimées. Les données sont stockées dans une seule table. On peut donc utiliser BeautifulSoup pour parser le code HTML, et naviguer entre les différents tags qui constituent la table. La méthode pandas pd.read_html() peut aussi nous simplifier la tâche, et est donc préférable ici.

** Methode 1 : avec pandas **

In [36]:
import pandas as pd

In [37]:
depression = pd.read_html('https://en.wikipedia.org/wiki/Epidemiology_of_depression')[0]
depression

,Category,1990,2017,Absolute change,Relative change
0,Afghanistan,439.81,443.57,+3.75,<1%
1,Albania,306.48,308.44,+1.96,<1%
2,Algeria,454.02,455.36,+1.34,<1%
3,American Samoa,299.00,301.44,+2.43,<1%
4,Andean Latin America,395.14,395.61,+0.47,<1%
...,...,...,...,...,...
226,Western Sub-Saharan Africa,269.65,272.36,+2.70,+1%
227,World,348.53,345.69,–2.84,>–1%
228,Yemen,442.49,446.34,+3.85,<1%
229,Zambia,334.01,337.31,+3.30,<1%


La fonction donne une liste de plusieurs tables tirées de la page - on a uniquement besoin de la première.

** Méthode 2 avec BeautifulSoup **

On télécharge la page avec requests.get puis on la parse.

In [38]:
import requests
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/Epidemiology_of_depression')
print(page)
soup = BeautifulSoup(page.content)

<Response [200]>


In [39]:
table = soup.find('table') # on veut le premier tag 'table'

On peut voir dans l'inspecteur que les lignes du tableau sont contenues dans les tags <tr> (pour "table row"), qui eux mêmes contiennent des tags <td> ("table data"). Il y a trois tags <td> par ligne, correspondant aux trois colonnes. On va donc regarder le tableau ligne par ligne et extraire les informations des tags <td>. On les stocke au fur et à mesure dans 3 listes.

In [40]:
# Récupérer toutes les lignes
rows = table.find_all('tr')

# Initialiser les listes
ranks = []
rates = []
countries = []
links = []

In [41]:
# Boucler sur les lignes pour extraire les données des cellules

for row in rows[1:]: # On commence juste après la ligne header

    cells = row.find_all('td')

    rank = cells[0]
    ranks.append(rank.text)

    country = cells[1]
    countries.append(country.text)

    rate = cells[2]
    rates.append(rate.text)

Reste le formating :

In [42]:
from unidecode import unidecode

countries = [unidecode(str(country)).strip() for country in countries]

rates = [float(rates[0].replace(',','')) for rate in rates]

On peut enfin reconstruire la table :

In [43]:
depression2 = pd.DataFrame(ranks, index=countries, columns=['Rank'])
depression2['DALY rate'] = rates
depression2.head()

,Rank,DALY rate
439.81,Afghanistan\n,443.57
306.48,Albania\n,443.57
454.02,Algeria\n,443.57
299.00,American Samoa\n,443.57
395.14,Andean Latin America\n,443.57


**Ensoleillement**

On effectue le même travail sur l'autre article qui compte les heures de soleil par pays :

In [44]:
sun = pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_by_sunshine_duration')
sun

[    0                                                  1
 0 NaN  This article may be too long to read and navig...,
                Country          City    Jan    Feb    Mar    Apr    May  \
 0          Ivory Coast        Gagnoa  183.0  180.0  196.0  188.0  181.0   
 1          Ivory Coast        Bouaké  242.0  224.0  219.0  194.0  208.0   
 2          Ivory Coast       Abidjan  223.0  223.0  239.0  214.0  205.0   
 3          Ivory Coast       Odienné  242.0  220.2  217.3  214.7  248.8   
 4          Ivory Coast         Ferké  279.0  249.0  253.0  229.0  251.0   
 ..                 ...           ...    ...    ...    ...    ...    ...   
 106      Guinea-Bissau        Bissau  248.0  226.0  279.0  270.0  248.0   
 107  Equatorial Guinea          Bata  201.5  192.1  173.6  177.0  189.1   
 108  Equatorial Guinea        Malabo  148.8  152.5  108.5  120.0  117.8   
 109            Namibia  Keetmanshoop  353.0  300.0  312.0  306.0  304.0   
 110            Namibia      Windhoek  288.0  2

In [45]:
len(sun)

8

Nous avons un tableau par continent. On les concatène donc tous en un seul. Tous les tableaux de données partagent les mêmes colonnes, la concaténation est donc facile.

In [46]:
sun = pd.concat(sun[1:]) # on saute le premier tableau, qui est vide
sun.reset_index()

,index,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year,Ref.,".mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vteClimate of the United States",".mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vteClimate of the United States.1"
0,0,Ivory Coast,Gagnoa,183.0,180.0,196.0,188.0,181.0,118.0,97.0,80.0,110.0,155.0,171.0,164.0,1823.0,[2],NaN,NaN
1,1,Ivory Coast,Bouaké,242.0,224.0,219.0,194.0,208.0,145.0,104.0,82.0,115.0,170.0,191.0,198.0,2092.0,[2],NaN,NaN
2,2,Ivory Coast,Abidjan,223.0,223.0,239.0,214.0,205.0,128.0,137.0,125.0,139.0,215.0,224.0,224.0,2296.0,[2],NaN,NaN
3,3,Ivory Coast,Odienné,242.0,220.2,217.3,214.7,248.8,221.8,183.5,174.5,185.4,235.8,252.0,242.6,2638.6,[3],NaN,NaN
4,4,Ivory Coast,Ferké,279.0,249.0,253.0,229.0,251.0,221.0,183.0,151.0,173.0,245.0,261.0,262.0,2757.0,[2],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,States,Alabama Alaska Arizona Arkansas California Col...
393,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Federal district,"Washington, D.C."
394,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insular areas,American Samoa Guam Northern Mariana Islands P...
395,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"By city, county, or region",Allentown Anchorage Beaumont Bismarck Chicago ...


On voit qu'il existe des données pour plusieurs villes par pays, et 1 mesure par mois. On calcule une simple moyenne annuelle d'ensoleillement par pays.

In [47]:
sun_country = sun.groupby('Country').mean()
sun_country

<ipython-input-47-2972f0768d9e>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sun_country = sun.groupby('Country').mean()


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Country,,,,,,,,,,,,,
Afghanistan,177.200000,178.600000,204.500000,232.500000,310.300000,353.400000,356.800000,339.70,303.900000,282.600000,253.200000,182.400000,3175.100000
Albania,124.000000,125.000000,165.000000,191.000000,263.000000,298.000000,354.000000,327.00,264.000000,218.000000,127.000000,88.000000,2544.000000
Algeria,223.300000,220.250000,262.200000,292.500000,323.800000,312.000000,353.250000,325.35,262.500000,269.500000,225.000000,204.400000,3266.500000
Angola,219.000000,208.000000,213.000000,199.000000,233.000000,223.000000,175.000000,150.00,145.000000,164.000000,199.000000,212.000000,2341.000000
Argentina,238.250000,207.066667,185.050000,170.616667,151.466667,123.383333,138.616667,170.40,174.600000,208.566667,222.833333,229.450000,2220.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uzbekistan,117.800000,127.100000,164.300000,216.000000,303.800000,363.000000,384.400000,365.80,300.000000,226.300000,150.000000,105.400000,2823.900000
Venezuela,259.133333,243.900000,261.700000,224.666667,217.400000,212.333333,248.100000,248.00,245.833333,251.400000,239.000000,243.933333,2895.300000
Vietnam,178.250000,168.000000,190.500000,185.000000,203.500000,182.250000,196.250000,175.00,161.750000,158.500000,157.250000,167.000000,2123.500000
